In [1]:
# Check the version of available sprak here -> https://downloads.apache.org/spark/
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q https://www-us.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Data Download

In [2]:
!wget -q https://s3-eu-west-1.amazonaws.com/csparkdata/ol_cdump.json -O /content/ol_cdump.json

Loading The Data

In [3]:
from pyspark.sql.functions import *

df = spark.read.format('json').option('multiline','false').load('/content/ol_cdump.json')

df.show(2,truncate=False)

+---------------+--------------------------+----+----------+----------------------+-------------+------------+--------------+------+--------------------------------------------+----------+-----------+-------------------+------------+------------+------------+--------+------------------+--------------+----------+-----------+------+---------+------------+-----------+-------+-------+------------+---------------+------------------+------------------+--------------------------------------------+---------------+------------------+----+-----+--------+----+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----+-----------+------------+---------------------------------------+----------------+-------------+------+-------------------+---------------+---------------+------------+--------------+----------------------------+----------

Cleansing Data

In [4]:
# Publish Date Greater than 1950, Title cannot be null or empty and authors cannot be null
# PUblish Date cannot be greater than current Year
df = df.where(col('publish_date')>1950).where(col('number_of_pages')>20).where("title is not null and title !='' and authors is not null")
df.show(10)

+---------------+--------------------+----+----------+--------------------+--------------------+------------+--------------+------+--------------------+----------+-----------+-------------------+------------+------------+-------------+--------+------------------+--------------+----------+-----------+--------------------+---------+------------+-----------+-------+-------+------------+---------------+------------------+------------------+--------------------+---------------+------------------+----+-----+--------+----+--------------------+---------------+-----+-----------+------------+--------------------+--------------------+-------------+------+-------------------+---------------+---------------+------------+-------------------+--------------------+------------+--------+--------------------+--------------+--------------+-------------+--------------+------------+-------------+--------------------+--------------------+-----------------+--------------------+------------+---------------+---

In [5]:
df.count()

75354

Select Harry Potter Books

In [6]:
# For selecting, we will utilize the select and where function
df.where(upper(col("title")).like("%HARRY POTTER%")).select("title").show(10, truncate = False)

+----------------------------------------+
|title                                   |
+----------------------------------------+
|Harry Potter and the philosopher's stone|
|Harry Potter y la piedra filosofal      |
+----------------------------------------+



Book With Most Pages

In [7]:
# For selecting book with maximum pages, we will utilize group by, max and sort 
max_page = df.groupBy().max("number_of_pages").select(col("max(number_of_pages)").alias("number_of_page"))
joinExpression = df["number_of_pages"] == max_page["number_of_page"]

df.join(max_page,joinExpression,"inner").select("*").show(5)

#df.where("number_of_pages=48418").select("*").show()

+---------------+--------------------+----+----------+--------------------+-------------+------------+--------------+------+--------------------+----------+-----------+-------------------+------------+------------+--------------+--------+------------------+--------------+----------+-----------+------+---------+------------+-----------+------------+---------------+------------+---------------+------------------+------------------+--------------------+---------------+-------------------+------------+-----+--------+----+--------------------+---------------+-----+-----------+------------+------------+--------------+-------------+------+-------------------+---------------+---------------+------------+--------------+--------------------+------------+--------+------+--------------+--------------+-------------+--------------+------------+-------------+--------------------+--------+-----------------+--------------------+------------+---------------+----------------+----+----+-------+------+----

Top 5 Authors

In [8]:
#df.where("authors[1] is not null").selectExpr("authors").show(10, truncate = False)
df.selectExpr("authors[0] as authors").groupBy("authors").count().orderBy(col("count").desc()).limit(5).show(truncate = False)

+------------------------+-----+
|authors                 |count|
+------------------------+-----+
|[, /authors/OL1224818A,]|236  |
|[, /authors/OL4283462A,]|116  |
|[, /authors/OL785848A,] |106  |
|[, /authors/OL1926829A,]|80   |
|[, /authors/OL883775A,] |75   |
+------------------------+-----+



Top 5 Genres

In [9]:
df.where("genres is not null").groupBy("genres").count().orderBy(col("count").desc()).limit(5).show(10, truncate = False)

+----------------------+-----+
|genres                |count|
+----------------------+-----+
|[Fiction.]            |2824 |
|[Biography.]          |2038 |
|[Juvenile literature.]|1145 |
|[Exhibitions.]        |774  |
|[Guidebooks.]         |424  |
+----------------------+-----+



Avg Number Of Pages

In [ ]:
df.groupBy().avg("number_of_pages").show()

Number of Authors per publish year

In [ ]:
df.selectExpr("authors[0] as authors","publish_date").groupBy("publish_date").agg(countDistinct("authors")).show(10)